In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.4684975020271792
Gradient Descent(2/49): loss=0.4513712748460005
Gradient Descent(3/49): loss=0.44178360296424807
Gradient Descent(4/49): loss=0.43617479340076165
Gradient Descent(5/49): loss=0.4326884948512339
Gradient Descent(6/49): loss=0.4303539841023341
Gradient Descent(7/49): loss=0.4286614226647361
Gradient Descent(8/49): loss=0.4273411475179239
Gradient Descent(9/49): loss=0.4262489193466734
Gradient Descent(10/49): loss=0.4253062112549615
Gradient Descent(11/49): loss=0.4244691000973839
Gradient Descent(12/49): loss=0.42371203005243574
Gradient Descent(13/49): loss=0.42301931548757576
Gradient Descent(14/49): loss=0.4223806759656577
Gradient Descent(15/49): loss=0.42178887555037825
Gradient Descent(16/49): loss=0.42123846301759116
Gradient Descent(17/49): loss=0.42072509010349285
Gradient Descent(18/49): loss=0.4202451348565581
Gradient Descent(19/49): loss=0.4197954872779463
Gradient Descent(20/49): loss=0.419373

In [ ]:
# Least squares regression using normal equation
weight, loss = least_squares(y, tX)
loss

In [ ]:
# Ridge regression using normal equations
lambda_ = 1e-3
weight, loss = ridge_regression(y, tX, lambda_)
loss

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)